In [45]:
import sympy as sp
import time
from tabulate import tabulate

In [46]:
#given values hot fluid: lube oil, cold fluid: crude oil
values = {
    'M_c': 72500, #lp/h
    'M_h': 6900, #lp/h, 
    'Cp_h': 0.62, #Btu/(lb degF)
    'Cp_c': 0.585, #Btu/(lb degF)
    'T_hi': 450, #degF
    'T_ci': 300 ,#degF 
    'U':28.2, #Btu/(hr ft2 degF)
    'A': 173 #ft2
}

In [47]:
#handle absurd values

if(values['T_hi'] < values['T_ci']):
    print("Hot fluid temperature is lower than cold fluid temperature. Please check the values.")
    exit()

for key in values:
    if(values[key] <= 0):
        print("Unexpected value found for " + key + ". Please check the values.")
        exit()

In [48]:
T_hi, T_ci, T_co, M_c, M_h, Cp_c, Cp_h, U, A = sp.symbols('T_hi T_ci T_co M_c M_h Cp_c Cp_h U A', real=True)

In [49]:
T_ho = T_hi - (M_c*Cp_c*(T_co-T_ci)/(M_h*Cp_h))
T_ho

-Cp_c*M_c*(-T_ci + T_co)/(Cp_h*M_h) + T_hi

In [50]:
def lmtd(delT1, delT2):
    return (delT1-delT2)/sp.log(delT1/delT2)

### co-current


In [51]:
delT1 = T_hi - T_ci
delT2 = T_ho-T_co

In [52]:
f = U*A*lmtd(delT1,delT2) - M_h*Cp_h*(T_hi-T_ho)
f

A*U*(Cp_c*M_c*(-T_ci + T_co)/(Cp_h*M_h) - T_ci + T_co)/log((-T_ci + T_hi)/(-Cp_c*M_c*(-T_ci + T_co)/(Cp_h*M_h) - T_co + T_hi)) - Cp_c*M_c*(-T_ci + T_co)

In [53]:
#subsituting values

f = f.subs([
    (M_c,values['M_c']),
    (M_h, values['M_h']),
    (Cp_c, values['Cp_c']),
    (Cp_h, values['Cp_h']),
    (T_hi, values['T_hi']),
    (T_ci, values['T_ci']),
    (U, values['U']),
    (A, values['A']),
]).evalf()
f

-42412.5*T_co + 4878.6*(10.914095371669*T_co - 3274.2286115007)/log(150/(3424.2286115007 - 10.914095371669*T_co)) + 12723750.0

In [54]:
#differentiating

df_dTco = sp.diff(f, T_co)
df_dTco

-42412.5 + 53245.5056802244/log(150/(3424.2286115007 - 10.914095371669*T_co)) - 0.681159332966027*(22.828190743338 - 0.0727606358111267*T_co)*(10.914095371669*T_co - 3274.2286115007)/((1 - 0.00318731504520833*T_co)**2*log(150/(3424.2286115007 - 10.914095371669*T_co))**2)

In [55]:
itr_table = []
def NR(fn, dfn_dTco, T_co_prev, tol, itr=1):
    decimal_places = int(-sp.log(tol, 10).evalf()+1)

    # NR definition: Xk+1 = Xk - f(k)/f'(k)
    T_co_next = (T_co_prev - fn/dfn_dTco).subs(T_co, T_co_prev).round(decimal_places)
    err = abs(T_co_next - T_co_prev)
    # store the iteration in table
    itr_table.append([itr,
                      T_co_prev,
                      T_co_next,
                      fn.subs(T_co, T_co_prev).round(6),
                      fn.subs(T_co, T_co_next).round(decimal_places),
                      err])

    # if less than tolerance then return
    if err < tol:
        return T_co_next

    # # else go for next iteration
    itr += 1
    return NR(fn, dfn_dTco, T_co_next, tol, itr)
 


In [56]:
itr_table = [['itr', 'Vm_prev', 'Vm_next',
              'fn(Vm_prev)', 'fn(Vm_next)', 'error']]
initial_guess = 301
tolerance = 1e-3
start = time.time()

#try except block to handle abstard values
try:
  result_NR = NR(f, df_dTco, initial_guess, tolerance)
  end = time.time()
  print('T_co:', str(result_NR)+' degF')
  print(f'Time taken: {end-start} second')
  print(tabulate(itr_table, headers='firstrow', tablefmt='fancy_grid'))

except:
  print('Error: T_co is not found. Please check the values entered.')
  exit()

T_co: 309.8273 degF
Time taken: 0.07142233848571777 second
╒═══════╤═══════════╤═══════════╤═══════════════╤═══════════════╤═════════════╕
│   itr │   Vm_prev │   Vm_next │   fn(Vm_prev) │   fn(Vm_next) │       error │
╞═══════╪═══════════╪═══════════╪═══════════════╪═══════════════╪═════════════╡
│     1 │   301     │   310.501 │  662420       │   -58237.3    │ 9.5013      │
├───────┼───────────┼───────────┼───────────────┼───────────────┼─────────────┤
│     2 │   310.501 │   309.841 │  -58237.3     │    -1175.09   │ 0.660099    │
├───────┼───────────┼───────────┼───────────────┼───────────────┼─────────────┤
│     3 │   309.841 │   309.827 │   -1175.09    │       -4.6556 │ 0.0138016   │
├───────┼───────────┼───────────┼───────────────┼───────────────┼─────────────┤
│     4 │   309.827 │   309.827 │      -4.65557 │        3.7524 │ 9.91821e-05 │
╘═══════╧═══════════╧═══════════╧═══════════════╧═══════════════╧═════════════╛


In [57]:
#result from NR is T_co
T_co_co_curr = T_co.subs(T_co, result_NR)

#subsituting T_co in T_ho to evaulate T_ho
T_ho_co_curr = T_ho.subs([
  (M_c,values['M_c']),
  (M_h, values['M_h']),
  (Cp_c, values['Cp_c']),
  (Cp_h, values['Cp_h']),
  (T_hi, values['T_hi']),
  (T_ci, values['T_ci']),
  (T_co, T_co_co_curr),
]).evalf().round(int(-sp.log(tolerance, 10).evalf()+1))


In [58]:
print('For co-current flow, T_co =', T_co_co_curr, 'degF', 'and T_ho =', T_ho_co_curr, 'degF')

For co-current flow, T_co = 309.8273 degF and T_ho = 352.5712 degF


### Counter-Current


In [59]:
delT1 = T_hi - T_co
delT2 = T_ho-T_ci

In [60]:
f = U*A*lmtd(delT1,delT2) - M_h*Cp_h*(T_hi-T_ho)
f

A*U*(Cp_c*M_c*(-T_ci + T_co)/(Cp_h*M_h) + T_ci - T_co)/log((-T_co + T_hi)/(-Cp_c*M_c*(-T_ci + T_co)/(Cp_h*M_h) - T_ci + T_hi)) - Cp_c*M_c*(-T_ci + T_co)

In [61]:
#subsituting values

f = f.subs([
    (M_c,values['M_c']),
    (M_h, values['M_h']),
    (Cp_c, values['Cp_c']),
    (Cp_h, values['Cp_h']),
    (T_hi, values['T_hi']),
    (T_ci, values['T_ci']),
    (U, values['U']),
    (A, values['A']),
]).evalf()
f

-42412.5*T_co + 4878.6*(8.914095371669*T_co - 2674.2286115007)/log((450 - T_co)/(3124.2286115007 - 9.914095371669*T_co)) + 12723750.0

In [62]:
#differentiating

df_dTco = sp.diff(f, T_co)
df_dTco

-42412.5 + 43488.3056802244/log((450 - T_co)/(3124.2286115007 - 9.914095371669*T_co)) - 4878.6*(3124.2286115007 - 9.914095371669*T_co)*(8.914095371669*T_co - 2674.2286115007)*(-1/(3124.2286115007 - 9.914095371669*T_co) + 1.01570474606413e-6*(450 - T_co)/(1 - 0.00317329382849062*T_co)**2)/((450 - T_co)*log((450 - T_co)/(3124.2286115007 - 9.914095371669*T_co))**2)

In [63]:
itr_table = [['itr', 'Vm_prev', 'Vm_next',
              'fn(Vm_prev)', 'fn(Vm_next)', 'error']]
initial_guess = 301
tolerance = 1e-3
start = time.time()

#try except block to handle abstard values
try:
  result_NR = NR(f, df_dTco, initial_guess, tolerance)
  end = time.time()
  print('T_co:', str(result_NR)+' degF')
  print(f'Time taken: {end-start} second')
  print(tabulate(itr_table, headers='firstrow', tablefmt='fancy_grid'))

except:
  print('Error: T_co is not found. Please check the values entered.')
  exit()
    

T_co: 310.0676 degF
Time taken: 0.057022809982299805 second
╒═══════╤═══════════╤═══════════╤═══════════════╤═══════════════╤════════════╕
│   itr │   Vm_prev │   Vm_next │   fn(Vm_prev) │   fn(Vm_next) │      error │
╞═══════╪═══════════╪═══════════╪═══════════════╪═══════════════╪════════════╡
│     1 │   301     │   310.534 │ 662531        │ -37276.6      │ 9.5341     │
├───────┼───────────┼───────────┼───────────────┼───────────────┼────────────┤
│     2 │   310.534 │   310.071 │ -37276.6      │   -301.768    │ 0.4627     │
├───────┼───────────┼───────────┼───────────────┼───────────────┼────────────┤
│     3 │   310.071 │   310.068 │   -301.768    │     -0.516403 │ 0.00379944 │
├───────┼───────────┼───────────┼───────────────┼───────────────┼────────────┤
│     4 │   310.068 │   310.068 │     -0.516396 │     -0.516403 │ 0          │
╘═══════╧═══════════╧═══════════╧═══════════════╧═══════════════╧════════════╛


In [64]:
T_co_cn_curr = T_co.subs(T_co, result_NR)

T_ho_cn_curr = T_ho.subs([
  (M_c,values['M_c']),
  (M_h, values['M_h']),
  (Cp_c, values['Cp_c']),
  (Cp_h, values['Cp_h']),
  (T_hi, values['T_hi']),
  (T_ci, values['T_ci']),
  (T_co, T_co_cn_curr),
]).evalf().round(int(-sp.log(tolerance, 10).evalf()+1))

In [65]:
print('For counter-current flow, T_co =', T_co_cn_curr, 'degF', 'and T_ho =', T_ho_cn_curr, 'degF')

For counter-current flow, T_co = 310.0676 degF and T_ho = 350.1889 degF


### Final Result


In [66]:
result = [
        ['Flow', 'T_hi (degF)', 'T_ci (fegF)', 'T_co (degF)', 'T_ho (degF)'],
        ['co-current', values['T_hi'], values['T_ci'], T_co_co_curr, T_ho_co_curr],
        ['counter-current', values['T_hi'], values['T_ci'], T_co_cn_curr, T_ho_cn_curr]
        ]

print(tabulate(result, headers='firstrow', tablefmt='fancy_grid'))

╒═════════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Flow            │   T_hi (degF) │   T_ci (fegF) │   T_co (degF) │   T_ho (degF) │
╞═════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ co-current      │           450 │           300 │       309.827 │       352.571 │
├─────────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ counter-current │           450 │           300 │       310.068 │       350.189 │
╘═════════════════╧═══════════════╧═══════════════╧═══════════════╧═══════════════╛
